In [133]:
%reset -f

In [134]:
import os
import pandas as pd
import xlwings as xw
import numpy as np

In [135]:
# Enter Client-Specific Parameters: key word for name, ISL

#client = "Client_Tech" 
client = "Client_Scale" 
client_ISL = 75000

In [136]:
# Set path, client name, find client files
base_path = '/Users/morgangodley/Documents/Projects Folder/Dummy Data'
client_files = [f for f in os.listdir(base_path) if client.lower() in f.lower() and f.endswith(".xls")]

# Print whether client files were found
if not client_files:
    print(f"No .xls files found for client: {client}")
else:
    print(f"Found file(s) for {client}: {client_files}")

    # Manipulate the First File found; use xlwings (xw) package to manipulate .xls documents
    client_file1 = client_files[0]
    client_final_xls_path = os.path.join(base_path, client_file1) # Reset path to First File
    client_final_xls = xw.Book(client_final_xls_path)             # Open file with xw package
    
    # Pull Experience & Large Claims YTD tabs
    for tab in client_final_xls.sheets:
        tab_name = tab.name
        # Pull Experience tab into DF df_exp
        if tab_name.startswith("EXP-"):
            df_exp = tab.used_range.options(pd.DataFrame, header=0, index=False).value
            print(f"\nExperience Sheet: {tab_name}, saved as DataFrame df_exp")
        # Pull Large Claims tab into DF df_lc
        elif tab_name.startswith("LC-"):
            df_lc = tab.used_range.options(pd.DataFrame, header=0, index=False).value
            print(f"\nLarge Claims Sheet: {tab_name}, saved as DataFrame df_lc")

    client_final_xls.close()

Found file(s) for Client_Scale: ['Client_Scale.xls']

Experience Sheet: EXP-CLIENT_SCALE AS-1, saved as DataFrame df_exp

Large Claims Sheet: LC-CLIENT_SCALE ASS-1, saved as DataFrame df_lc


<h1> Large Claims YTD </h1>

In [137]:
df_lc.head(15)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,CLIENT SCALE,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,DE-IDENTIFIED CLAIMS EXCEEDING REPORT,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,JAN 2025 thru MARCH 2024,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,RATING TYPE: POOLED STOP LOSS,None,None,None,None,None,None,None,None,None,None,None,None,None,None
5,ACCOUNT: 061111-SCALE,None,None,None,None,None,None,None,None,None,None,None,None,None,None
6,"CLAIMS EXCEEDING: $25,000.00",None,None,None,None,None,None,None,None,None,None,None,None,None,None
7,"REPORTED CLAIMS: INNET, OUTNET, DRUG",None,None,None,None,None,None,None,None,None,None,None,None,None,None
8,DIAGNOSIS IS BASED ON LARGEST MEDICAL CLAIM AM...,None,None,None,None,None,None,None,None,None,None,None,None,None,None
9,MEMBER STATUS IS …,None,None,None,None,None,None,None,None,None,None,None,None,None,None


Prepare Large Claims DF

In [138]:
# Remove Cigna's header
df_lc_cleaned = df_lc.iloc[10:].reset_index(drop=True)         # Drop first 10 rows
df_lc_cleaned.columns = df_lc_cleaned.iloc[0]                  # Row 11 becomes headers
df_lc_cleaned = df_lc_cleaned.iloc[1:].reset_index(drop=True)  # Drop the header row

# Convert $ columns to numeric
cols_convert = ["DRUG CLAIMS", "PAID CLAIMS", "CLAIMANT TOTAL"]

for col in cols_convert:
    df_lc_cleaned[col] = (
        df_lc_cleaned[col]
        .astype(str)                             # convert to string
        #.str.replace(r"[\$,]", "", regex=True)  # remove $ and commas
        .replace("None", np.nan)                 # convert 'None' strings to np.nan
        .astype(float)                           # convert to float
    )

# Dominate Diagnosis per Member
dominant_icd = (df_lc_cleaned.loc
                # Find the largest CLAIMANT TOTAL for each MEMBER ID
                [df_lc_cleaned.groupby("MEMBER ID")["CLAIMANT TOTAL"].idxmax()]
                # Select corresponding ICD DESCRIPTION for max CLAIMANT TOTAL
                .set_index("MEMBER ID")["ICD DESCRIPTION"])
# Replace all ICD DESCRIPTION values by MEMBER ID with dominant ICD code
df_lc_cleaned["ICD DESCRIPTION"] = df_lc_cleaned["MEMBER ID"].map(dominant_icd)



In [139]:
df_lc_cleaned

,BENEFIT OPTION,FUNDING TYPE,RATING TYPE,MEMBER ID,MEMBER STATUS,REL,GENDER,AGE BAND,ICD CODE,ICD DESCRIPTION,ICD VERSION,LAST DATE OF SERVICE,DRUG CLAIMS,PAID CLAIMS,CLAIMANT TOTAL
0,MOAP0061-OAP BUY-UP,1.0,M,11111.0,COBRA,SB,M,30-39,*,UNSPECIFIED,*,2025-03-15 00:00:00,0.00,300000.92,300000.92
1,MOAP0061-OAP BUY-UP,2.0,R,11111.0,COBRA,SB,M,30-39,A11,UNSPECIFIED,10.0,2025-03-15 00:00:00,3500.00,50000.10,53500.10
2,None,None,None,MEMBER ID Total,None,None,None,None,None,None,None,None,3500.00,350001.02,353501.02
3,None,None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,NaN
4,MOAP0061-OAP BUY-UP,2.0,R,22222.0,ACTIVE,CH,F,1-17,*,UNSPECIFIED,*,2025-03-01 00:00:00,50.00,70000.55,70050.55
5,None,None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,NaN
6,MOAP0061-OAP BUY-UP,1.0,M,33333.0,ACTIVE,SP,F,50-59,*,AUTOIMMUNE1,*,2025-03-20 00:00:00,0.30,15000.00,15000.30
7,MOAP0061-OAP BUY-UP,2.0,R,33333.0,ACTIVE,SP,F,50-59,A22,AUTOIMMUNE1,10.0,2025-03-20 00:00:00,0.00,80000.00,80000.00
8,None,None,None,MEMBER ID Total,None,None,None,None,None,None,None,None,0.30,95000.00,95000.30
9,None,None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,NaN


Subset Large Claims

In [140]:
# Keep Relevant Columns: MEMBER ID, REL, ICD DESCRIPTION, DRUG CLAIMS, PAID CLAIMS
df_lc_cleaned_subset = df_lc_cleaned[["MEMBER ID", "MEMBER STATUS", "REL", "ICD DESCRIPTION", 
                                      "DRUG CLAIMS", "PAID CLAIMS", "CLAIMANT TOTAL"]].copy()

# Drop rows with "MEMBER ID Total" in MEMBER ID column
df_lc_cleaned_subset = df_lc_cleaned_subset[df_lc_cleaned_subset["MEMBER ID"] != "MEMBER ID Total"]
df_lc_cleaned_subset = df_lc_cleaned_subset.dropna(subset=["MEMBER ID"]).reset_index(drop=True)

# Group on MEMBER ID, sum numeric columns, for non-numeric columns, keep the first value
df_lc_cleaned_subset = df_lc_cleaned_subset.groupby("MEMBER ID").agg(
    {"MEMBER STATUS": "first",
     "REL": "first",
     "ICD DESCRIPTION": "first",
     "DRUG CLAIMS": "sum",
     "PAID CLAIMS": "sum",
     "CLAIMANT TOTAL": "sum"}).reset_index()

#Re-order columns
df_lc_cleaned_subset = df_lc_cleaned_subset[
    ["MEMBER ID", "REL", "MEMBER STATUS", "ICD DESCRIPTION", 
     "PAID CLAIMS", "DRUG CLAIMS", "CLAIMANT TOTAL"]]

# Sort CLAIMANT TOTAL in descending order
df_lc_cleaned_subset = df_lc_cleaned_subset.sort_values("CLAIMANT TOTAL", ascending=False).reset_index(drop=True)

# Keep Claims only >= 50k
# df_lc_cleaned_subset = df_lc_cleaned_subset[df_lc_cleaned_subset["CLAIMANT TOTAL"] >= 50000]


In [141]:
# ISL - medical claims-only
df_lc_cleaned_subset["ISL"] = np.where(
    df_lc_cleaned_subset["PAID CLAIMS"] > client_ISL,
    df_lc_cleaned_subset["PAID CLAIMS"] - client_ISL,
    np.nan
)

# Round numeric columns to 2 decimal places
df_lc_cleaned_subset[["PAID CLAIMS", "DRUG CLAIMS", "CLAIMANT TOTAL", "ISL"]] = (
    df_lc_cleaned_subset[["PAID CLAIMS", "DRUG CLAIMS", "CLAIMANT TOTAL", "ISL"]].round(2))


In [142]:
df_lc_cleaned_subset

,MEMBER ID,REL,MEMBER STATUS,ICD DESCRIPTION,PAID CLAIMS,DRUG CLAIMS,CLAIMANT TOTAL,ISL
0,11111.0,SB,COBRA,UNSPECIFIED,350001.02,3500.00,353501.02,275001.02
1,33333.0,SP,ACTIVE,AUTOIMMUNE1,95000.00,0.30,95000.30,20000.00
2,44444.0,SB,ACTIVE,ORTHO1,82000.30,900.55,82900.85,7000.30
3,55555.0,SP,ACTIVE,AUTOIMMUNE2,73000.55,60.00,73060.55,NaN
4,22222.0,CH,ACTIVE,UNSPECIFIED,70000.55,50.00,70050.55,NaN
5,66666.0,SB,COBRA,KIDNEY,45000.00,0.00,45000.00,NaN


<h3> Large Claims YTD - Final Output </h3>

In [143]:
df_lc_ytd_final = df_lc_cleaned_subset.drop(columns=["MEMBER ID"])

In [144]:
df_lc_ytd_final

,REL,MEMBER STATUS,ICD DESCRIPTION,PAID CLAIMS,DRUG CLAIMS,CLAIMANT TOTAL,ISL
0,SB,COBRA,UNSPECIFIED,350001.02,3500.00,353501.02,275001.02
1,SP,ACTIVE,AUTOIMMUNE1,95000.00,0.30,95000.30,20000.00
2,SB,ACTIVE,ORTHO1,82000.30,900.55,82900.85,7000.30
3,SP,ACTIVE,AUTOIMMUNE2,73000.55,60.00,73060.55,NaN
4,CH,ACTIVE,UNSPECIFIED,70000.55,50.00,70050.55,NaN
5,SB,COBRA,KIDNEY,45000.00,0.00,45000.00,NaN
